In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
tf.__version__

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

Reading Data

In [ ]:
PATH = os.getcwd()
data_dir = PATH + "/data/classification_data"
print(data_dir)

In [ ]:
batch_size = 32
img_height = 528
img_width = 652

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    image_size=(img_height, img_width),
    seed=123,
    batch_size=batch_size,
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    image_size=(img_height, img_width),
    seed=123,
    batch_size=batch_size,
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        aax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis('off')


In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)
print(tf.data.experimental.cardinality(val_ds))
print(tf.data.experimental.cardinality(test_ds))

Preprocessing Images

In [ ]:
def preprocess_image(img, lbl):
    img = tf.image.resize(img, [224, 224], method=tf.image.ResizeMethod.BILINEAR, antialias=True)
    return img, lbl

In [ ]:
train_ds = train_ds.map(preprocess_image)
val_ds = val_ds.map(preprocess_image)
# test_ds = test_ds.map(preprocess_image)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Model Declaration

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
len(list(base_model.layers))

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(8)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.summary()

In [ ]:
initial_epochs = 10

In [ ]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,
                    )

Tuning Model

In [ ]:
base_model.trainable = True

fine_tune_at = 180

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
    metrics=['accuracy'],
)

In [ ]:
model.summary()

In [ ]:
fine_tune_epochs = 30
total_epochs = initial_epochs + fine_tune_epochs
history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,)

In [ ]:
image_batch, label_batch = test_ds.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)


predictions = tf.nn.softmax(predictions, axis=-1)
predicted_classes = tf.argmax(predictions, axis=-1)

print('Predictions:\n', predicted_classes.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title(class_names[predicted_classes[i]] + class_names[label_batch[i]])
    plt.axis("off")

plt.show()

In [ ]:
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim(min(plt.ylim()), 1)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Sparse Categorical Cross Entropy Loss')
plt.ylim([[0, 1.0]])
plt.title("Training and Validation Loss")
plt.xlabel('epoch')
plt.show()

In [ ]:
# Confusion Matrix
cm = tf.math.confusion_matrix(label_batch, predicted_classes)

In [ ]:
cm

In [ ]:
cm_np = cm.numpy()

In [ ]:
sns.heatmap(cm_np, annot=True)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
model.save(os.getcwd() + '/models/classification.keras')

In [ ]:
m2 = tf.keras.models.load_model('saved_models/classification.keras')
# m2.layers[-2].trainable = False

In [ ]:
# import numpy as np
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 
# num_batches_test = 6
# batch_size = 32
# 
# total_predictions = []
# total_labels = []
# 
# for _ in range(num_batches_test):
#     # noinspection PyUnresolvedReferences
#     image_batch, label_batch = t_ds.as_numpy_iterator().next()
#     predictions = m2.predict_on_batch(image_batch)
#     predictions = tf.nn.softmax(predictions, axis=-1)
#     predicted_classes = tf.argmax(predictions, axis=-1)
#     
#     total_predictions.extend(predicted_classes.numpy().tolist())
#     total_labels.extend(label_batch.tolist())


In [ ]:
# y_true = total_labels
# y_pred = total_predictions
# 
# accuracy = accuracy_score(y_true, y_pred)
# precision = precision_score(y_true, y_pred, average='weighted')
# recall = recall_score(y_true, y_pred, average='weighted')
# f1 = f1_score(y_true, y_pred, average='weighted')

In [ ]:
# print('Accuracy:', accuracy)
# print('Precision:', precision)
# print('Recall:', recall)
# print('F1:', f1)

In [ ]:
image_batch, label_batch = test_ds.as_numpy_iterator().next()
predictions = m2.predict_on_batch(image_batch)

# predictions = tf.nn.softmax(predictions, axis=-1)
predicted_classes = tf.argmax(predictions, axis=-1)

print('Predictions:\n', predicted_classes.numpy())
print('Labels:\n', label_batch)


In [ ]:
for image, label in test_ds.take(1):
    print(image[0])

In [ ]:
image = tf.io.read_file("data/classification_data/Chaunsa (White)/IMG_20210705_100452.jpg")

In [ ]:
image = tf.image.decode_jpeg(image, channels=3)

In [ ]:
image = tf.image.resize(image, [224, 224], method=tf.image.ResizeMethod.BILINEAR, antialias=True)

In [ ]:
image = tf.convert_to_tensor(image, dtype=tf.float32)

In [ ]:
print(image)

In [ ]:
image = tf.expand_dims(image, axis=0)

In [ ]:
print(image)

In [ ]:
p = m2.predict_on_batch(image)

In [ ]:
p = tf.argmax(p, axis=-1)

In [ ]:
print(p.numpy()[0])